# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [9]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [4]:
# Following are shell commands to download the data
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv

!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv 

!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 57935  100 57935    0     0   152k      0 --:--:-- --:--:-- --:--:--  152k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   835  100   835    0     0   2392      0 --:--:-- --:--:-- --:--:--  2392
100   835  100   835    0     0   2392      0 --:--:-- --:--:-- --:--:--  2385
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   872  100   872    0     0   2408      0 --:

In [7]:
!Is

'Is' is not recognized as an internal or external command,
operable program or batch file.


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [10]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [11]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [12]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [13]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [16]:
merge = pd.merge(pop,abbrevs,how = 'outer', left_on='state/region',right_on='abbreviation')
merge.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


## Look for any rows with null values in the new dataframe

In [17]:
merge.isnull().values.any()

True

In [18]:
merge.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
abbreviation     True
dtype: bool

## Print out the rows in the merged dataframe that has null values for population.

In [19]:
merge[merge.isna().any(axis=1)].head(5)

,state/region,ages,year,population,state,abbreviation
2448,PR,under18,1990,NaN,NaN,NaN
2449,PR,total,1990,NaN,NaN,NaN
2450,PR,total,1991,NaN,NaN,NaN
2451,PR,under18,1991,NaN,NaN,NaN
2452,PR,total,1993,NaN,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [21]:
merge.loc[merge['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [22]:
merge.loc[merge['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merge.loc[merge['state/region'] == 'USA', 'state'] = 'United States'
merge.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
abbreviation     True
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [23]:
final = pd.merge(merge, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,AL,52423.0
1,AL,total,2012,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010,1130966.0,Alabama,AL,52423.0
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011,1125763.0,Alabama,AL,52423.0


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [24]:
final.isnull().values.any()

True

In [25]:
final.columns[final.isnull().any()]

Index(['population', 'abbreviation', 'area (sq. mi)'], dtype='object')

In [26]:
final[final.isna().any(axis=1)].head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
2448,PR,under18,1990,NaN,Puerto Rico,NaN,3515.0
2449,PR,total,1990,NaN,Puerto Rico,NaN,3515.0
2450,PR,total,1991,NaN,Puerto Rico,NaN,3515.0
2451,PR,under18,1991,NaN,Puerto Rico,NaN,3515.0
2452,PR,total,1993,NaN,Puerto Rico,NaN,3515.0


In [28]:
final.isnull().values.any()

True

## Select the portion of the data corresponding with the year 2000, and the total population.

In [29]:
final[final['year']==2000].groupby('year').sum()

,population,area (sq. mi)
year,,
2000,713976868.0,7580798.0


## Re-index the data on state and find population density and display by density in ascending order

In [30]:
final['state'].reindex()
final['density'] = final['population']/final['area (sq. mi)']
final.sort_values('density')

,state/region,ages,year,population,state,abbreviation,area (sq. mi),density
49,AK,under18,1990,177502.0,Alaska,AK,656425.0,0.270407
51,AK,under18,1991,182180.0,Alaska,AK,656425.0,0.277534
85,AK,under18,2008,183124.0,Alaska,AK,656425.0,0.278972
83,AK,under18,2007,184344.0,Alaska,AK,656425.0,0.280830
52,AK,under18,1992,184878.0,Alaska,AK,656425.0,0.281644
59,AK,under18,1995,184990.0,Alaska,AK,656425.0,0.281814
79,AK,under18,2005,185304.0,Alaska,AK,656425.0,0.282293
61,AK,under18,1996,185360.0,Alaska,AK,656425.0,0.282378
81,AK,under18,2006,185580.0,Alaska,AK,656425.0,0.282713
77,AK,under18,2004,186335.0,Alaska,AK,656425.0,0.283863


The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!